# SAO/LIP Python Primer Course Exercise Set 8

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/acorreia61201/SAOPythonPrimer/blob/main/exercises/Exercises8.ipynb)

## Exercise 1: The Gamma Function

In some previous exercises, we've used the *factorial*. For an integer $n$, its factorial $n!$ is the product of all positive integers before it. For example, $4! = 1 \cdot 2 \cdot 3 \cdot 4 = 24$.

Notice the emphasis on *integers*; the factorial is undefined for non-integer real numbers. We can generalize the concept of the factorial using the *gamma function*, defined below:

\begin{equation}
\Gamma (z) = \int_0^{\infty} t^{z-1}e^{-t} dt
\end{equation}

This function requires that $z > 0$ for real output. If $z$ is an integer, $\Gamma(z) = (z-1)!$. The definition of $\Gamma$ is generally a *nonelementary integral*, meaning we can't use calculus techniques to solve it (the exception is when $t$ is an integer). We can, however, use numerical techniques to approximate its value.

**Your task:** Write the definition of $\Gamma(z)$ as a function, setting $z$ to a value of your choice. Using `scipy.integrate.quad()`, integate this function from 0 to the upper bounds $(1, 5, 10, 50, 100, 1000, \infty)$ and print out their values. (Hint: You can use `numpy.inf` for the last one.)

Compare these values to the output of `scipy.special.gamma(z)` using the $z$ value you chose. How quickly does the integral converge to the true value? How accurate is the integral over $[0, \infty]$ to the gamma function?

In [ ]:
# YOUR CODE HERE

**Your task:** Now, let's compare the integrals to the function itself. Define an array of $z$ values ranging over $[0.01, 4.5]$, and do the same integrations as above by iterating over this array. One one axis, plot the integrals with dashed lines and the `scipy` gamma function with a solid black line. Label them accordingly. Also, plot the values of $1!$, $2!$, $3!$, and $4!$ as blue stars. How well do the functions match with the factorial values?

In [ ]:
# YOUR CODE HERE

As you can probably tell, the factorial is a very quickly growing function, making computational calculations very lengthy and possibly inaccurate. We can remedy this by taking the natural logarithm of the factorial, massively reducing the scale of the problem. For extremely large values, it may be more convenient to use the *Stirling approximation*, which states:

\begin{equation}
\ln(n!) \approx n \ln n - n
\end{equation}

**Your task:** Use `scipy.special.gammaln()` and `numpy.geomspace()` to calculate the factorials of an array of 500 values on the range $[1, 10^{20}]$. Plot the absolute error between these values and the Stirling approximation on a loglog plot:

\begin{equation}
| \ln \Gamma(x) - (x \ln x - x) |
\end{equation}

How does the error change with increasing values of $x$?

In [ ]:
# YOUR CODE HERE

## Exercise 2: Pendulum Motion

A pendulum is a good way to introduce oscillatory motion. The exact equation for the motion of a pendulum with mass $m$ and string length $L$ is:

\begin{equation}
m\frac{d^2\theta}{dt^2} + \frac{mg\sin(\theta)}{L} = 0
\end{equation}

Here, $\theta$ is the angle of the pendulum relative to its stable equilibrium position. That is, $\theta = 0$ corresponds to the pendulum hanging straight down. As always, $g=9.81 m/s^2$ is the gravitational constant. We're assuming that the pendulum has no friction.

This is a second-order differential equation, and a pretty difficult one at that. We can simplify this by taking into account the angular velocity of the pendulum, which we define as $\omega = \frac{d\theta}{dt}$. Thus, we can split this second-order equation into a system of two first-order equations:

\begin{align}
\frac{d\omega}{dt} &= - \frac{g\sin(\theta)}{L} \\
\frac{d\theta}{dt} &= \omega
\end{align}

**Your task:** Define a function that encodes the above system of equations. It should take in the initial conditions vector, the parameter $L$, and time as inputs.

In [ ]:
# YOUR CODE HERE

Let's say I have a pendulum with $L = 0.3$ meters. I drop the pendulum (i.e. release from rest) from an angular position $\theta_0 = \pi/4$.

**Your task:** Use `scipy.integrate.ivp_solve()` to solve the equations of motion for the system over 10 seconds. Make a figure showing the angular position and velocity of the pendulum over time in two separate plots and label them accordingly.

In [ ]:
# YOUR CODE HERE

**Your task:** Do the same as above, except now I release a pendulum with $L = 1.7$ meters from $30^o$. (Hint: You'll have to convert the angular value.) How do these plots differ from above? Try changing the initial conditions and parameters and see how they change.

In [ ]:
# YOUR CODE HERE

Even the above system of first-order equations is difficult to solve analytically. If we wanted to do this by hand, we can make an assumption that $\theta$ is small, and we can approximate $\sin(\theta) \approx \theta$ (this comes from the series approximation we've looked at before). The second-order equation then becomes:

\begin{equation}
m\frac{d^2\theta}{dt^2} + \frac{mg\theta}{L} = 0
\end{equation}

We can easily solve this using properties of trig functions to get an exact equation of motion solution:

\begin{equation}
\theta(t) = \theta_0\cos\bigg( \sqrt{\frac{g}{L}}t\bigg)
\end{equation}

**Your task:** Solve for the motion of the initial pendulum ($L=0.3$) under five scenarios: $(\theta_0, \omega_0) = (5^o, 0), (10^o, 0), (15^o, 0), (20^o, 0), (25^o, 0)$ over 10 seconds each. Plot the positions in each of these scenarios versus time in separate plots. In the same plots, overlay the analytic solution above with a dashed line. Title the plots accordingly by the initial angles. How do the numerical solutions compare to the analytic solutions in each case? What can we consider to be "small" for the approximation to hold?

In [ ]:
# YOUR CODE HERE

## Exercise 3: The SIR Model

The *SIR model* is a simple method of modelling the spread of an infectious disease in a population. It assumes that a population of size $N$ can be split into three groups that change in size over time as the disease spreads:

- the susceptible population $S(t)$, containing those who are able to contract the disease. We assume that everyone that hasn't contracted the disease can contract it.
- the infected population $I(t)$, containing those who currently have the disease and can spread it.
- the recovered population $R(t)$, containing those who have had the disease and have since become immune. We assume that there are no "carriers", i.e. that people who have recovered cannot spread the disease further. We also assume that the disease is non-lethal, so the population size doesn't change.

We model the infectivity of the disease with two parameters $\beta$ and $\gamma$. $\beta$ measures the *contact rate* of individuals in the population, such that each day the average person infects $\beta N$ other people. $\gamma$ measures the *mean recovery rate*, such that an infected person remains infective for $1/\gamma$ days.

We can model the change over time in the populations $S$, $I$, and $R$ using a system of differential equations:

\begin{align}
\frac{dS}{dt} &= -\frac{\beta SI}{N} \\
\frac{dI}{dt} &= \frac{\beta SI}{N} - \gamma I \\
\frac{dR}{dt} &= \gamma I
\end{align}

By supplying an initial condition vector $y = [S(0), I(0), R(0)]$, we can integrate the above conditions and get three curves that model the three populations over time.

**Your task:** Write a function that encodes the system of equations above. The function should take in the initial conditions vector, infectivity parameters, population size, and time as inputs.

In [ ]:
#YOUR CODE HERE

Let's start with a simple example. At a school with $N=1000$ initially healthy students and faculty, one student comes in with the flu. On average, an infected individual spreads flu to $\beta = 0.6$ other people, and this strain of flu has an average infectious period of $1/\gamma = 5$ days.

**Your task:** Set up the above scenario. Use `scipy.integrate.odeint()` to solve the system of equations using the function you defined above for 100 days. Plot the $S$, $I$, and $R$ curves, labelling each curve and the axes accordingly.

Play around with the parameters. What happens if $\beta$ increases and decreases? What about $\gamma$? Or $N$?

In [ ]:
# YOUR CODE HERE

Let's try a real-life example using COVID-19. Current models estimate that COVID-19 has a peak infectious period of $1/\gamma = 10$ days after symptoms start. Let's model a city that has $N = 1,000,000$ totally healthy people, assuming that no one moves in or out during that period. Of this population, let's say $I(0) = 10$ people return from a bus trip on which they contracted COVID-19.

We'll model three scenarios:

- About $10 \%$ of the population wears a mask and practices social distancing, which we'll assign an average contact rate of $\beta = 0.9$
- About half of the population wears a mask and social distances, which we'll assign $\beta = 0.5$
- About $90 \%$ of the population wears a mask and social distances, which we'll assign $\beta = 0.1$

We'll assume that repeat infections aren't a factor. We'll also assume that vaccines aren't in consideration; either this scenario was pre-2021 or this is some "super-variant" of COVID-19 that completely bypasses them.

**Your task:** Solve the system of differential equations for $200$ days. Plot these three scenarios in the same figure, titling each of them accordingly. Again, label the $S$, $I$, and $R$ curves and the axes accordingly.

Take note of the shapes of the curves. In which scenario does the number of infected peak the fastest, and which one the slowest? Which scenarios have the highest and lowest peak infected? How does the final number of recovered individuals change in the three scenarios?

In [ ]:
# YOUR CODE HERE